In [3]:
from embit import bip39, bip32
import os, hashlib

In [4]:
mnemonic = bip39.mnemonic_from_bytes(hashlib.sha256(b"random stuff").digest())
mnemonic

'carry decide decline search grant chest demand cram crane creek confirm ill limb text bar almost donor club prefer hand future hair endless proof'

In [5]:
seed = bip39.mnemonic_to_seed(mnemonic, '')
seed.hex()

'f7718b42e345f991289e3db39ca0f64ed8c25a2481d8bbd31cd972b96ea902dbca30c2c00fe631f279371343c1cbd0e71d6c1cde944ab62b26d058cd9b8e79bc'

In [6]:
root = bip32.HDKey.from_seed(seed)
root.to_base58()

'xprv9s21ZrQH143K3vWs32LN8d3nPPWywCw6DRtPZdb6qpZHQtuT1iDogKWcqT9EpRuEn5ivoTe7B8iNUza4A8527crvTKGhexMaG54494QjEaF'

In [7]:
root.serialize()[4:].hex()

'000000000000000000bb4dc15b22e4c399c49458bc45581d64133796673f3c4adfbda24a12d58bf6f500a971004c8f2e179bb31a2af499c39bc180e94bac65eee46ba0439632f14cc060'

In [9]:
from applets.core import get_connection, SecureAppletBase

class BlindOracle(SecureAppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CE01", connection)

    def set_seed(self, seed):
        return self.secure_request(b'\x10\x02'+seed)

    def derive_seed(self, password=b''):
        return self.secure_request(b'\x10\x04'+password)

    def get_xpub(self, derivation=b''):
        return self.secure_request(b'\x11\x01'+derivation)

In [10]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

In [11]:
app = BlindOracle(connection)
app.select()

b''

In [12]:
app.establish_secure_channel()
app.is_secure_channel_open

True

In [15]:
app.set_seed(seed)

b''

In [16]:
app.derive_seed()

b''

In [17]:
app.get_xpub().hex()

RuntimeError: Card returned secure error with code '0000' and data b'\x00\x00\x00\x00\x00\x00\x00\xbbM\xc1["\xe4\xc3\x99\xc4\x94X\xbcEX\x1dd\x137\x96g?<J\xdf\xbd\xa2J\x12\xd5\x8b\xf6\xf5\x03\xdb\x1afe\xe5\xbc\xca\xaaqT<\x18T\x9d\x87\xbb\xac~\x9a\x10\xd8\x0f\x0e\xf0\xc9\x92L\x8c\x08>\xc3R\x00\x00'

In [19]:
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbbM\xc1["\xe4\xc3\x99\xc4\x94X\xbcEX\x1dd\x137\x96g?<J\xdf\xbd\xa2J\x12\xd5\x8b\xf6\xf5\x03\xdb\x1afe\xe5\xbc\xca\xaaqT<\x18T\x9d\x87\xbb\xac~\x9a\x10\xd8\x0f\x0e\xf0\xc9\x92L\x8c\x08>\xc3R'.hex()

'000000000000000000bb4dc15b22e4c399c49458bc45581d64133796673f3c4adfbda24a12d58bf6f503db1a6665e5bccaaa71543c18549d87bbac7e9a10d80f0ef0c9924c8c083ec352'

In [21]:
root.to_public().serialize()[4:].hex()

'000000000000000000bb4dc15b22e4c399c49458bc45581d64133796673f3c4adfbda24a12d58bf6f503db1a6665e5bccaaa71543c18549d87bbac7e9a10d80f0ef0c9924c8c083ec352'

In [23]:
connection.disconnect()